> **DO NOT EDIT IF INSIDE `tsds` folder** 

# Week 5: Networks 2


*Thursday, March 8th, 2018*

Networks are mathematical representations of complex systems. We can use networks to gain various statistical insight about the system we're representing, but we can also use the network as a substrate on which to model phenomena and simulate processes. Some systems may have clusters: we can discover those by employing *community detection* algorithms. Other systems may be perfect for simulating how diseases spread. This week we will explore these two concepts. You will learn:

* How a popular community detection algorithm works.
* How null models are used in networks.
* How to simulate spreading on a network.

**Questions**: Outside of class, use [issue on GitHub](https://github.com/abjer/tsds/issues) for asking questions.

## Exercises

### Part 1: Community detection

Communities are little lumps of nodes in a network that are unusually strongly connected. Your family is a community, and your friend group from work or university is another community. While there is no one true definition of what a community is, there are many many different methods and algorithms for finding them. Here we will work with one of the most popular ones: [Louvain Modularity](https://en.wikipedia.org/wiki/Louvain_Modularity). The following exercises will walk you through the fundamentals of this method, and finally have you apply it to the network you used last week.

#### The modularity function

> **Ex. 5.1.1**: Assume you have a network where nodes and links tend to form lumps here and there. Imagine you now reach for your pen, and start labeling these nodes with group names (or ids) that feel appropriate according to how they are lumped together. If your partition is "good", nodes that are connected in groups should intuitively have the same label, while nodes that are distant and disconnected should have different labels. Modularity is a function that can be used to measure *how good* your partition is. It is in technical terms a *utility function*, and it looks like this:
> <br><br>
> $$ Q = \dfrac{1}{2m}\sum_{ij}{\left[A_{ij}-\dfrac{k_ik_j}{2m}\right]\delta(c_i, c_j)}.$$
> <br>
> Your job in this problem is to explain this equation. When I look at daunting math I tend to think of it as a program. Since all math can be implemented in code, all math can be broken into parts, where each part has it's own little job to perform. Answer each question below seperately:
1. In code, a sum, $\sum$, is like a `for` loop, where in every iteration you increment a variable. In the equation for modularity the little $ij$ subscript tells is what the sum is looping over (like `for ij in sumloop`). But what is $ij$?
2. In each iteration of the sum, the delta function $\delta(c_i, c_j)$ is used, where $c_i$ is the community label of node $i$. The delta function is a very simple program that returns 0 if the two input values are different and 1 if they are they same. How would you implement the delta function in code? What is it used for in the modularity equation?
3. Inside the sum we use the term $\frac{k_ik_j}{2m}$ as our *null model*. $k$ is the degree sequence (so $k_i$ is the degree of node $i$) and $m$ is the sum of all link weights. Explain what this null model measures. Could we have used other null models?
4. The sum subtracts the null model from $A_{ij}$ and adds the result to its final value if the delta function evaluates to 1. What is the point of only summing over this difference when the delta function is 1?
5. The sum term is normalized by $2m$. Why exactly $2m$?
6. Summarize your insight gained from answering the above questions. In your own words, explain how the modularity function works. Use 1-3 sentences.


> **Ex. 5.1.2**: Implement the modularity function. Write a Python function that takes as input an adjacency matrix and a label vector, and returns the modularity. Compute and print the modularity for the ones given below.

In [119]:
import numpy as np

A = np.array([
    [0, 1, 1, 0, 0, 0],
    [1, 0, 1, 0, 0, 0],
    [1, 1, 0, 1, 0, 0],
    [0, 0, 1, 0, 1, 1],
    [0, 0, 0, 1, 0, 1],
    [0, 0, 0, 1, 1, 0],
])

c = [0, 0, 0, 0, 1, 1]

def modularity(A, c):
    """Compute modularity for a labeled network.
    
    Input
    -----
        A : numpy.array
            Adjacency matrix. (N, N) square matrix.
        c : list of ints
            Community labels. Length N.
    
    Output
    ------
        out : float
    """
    # Your beautiful code goes here

> **Ex. 5.1.3**: The example labeling, `c`, was not optimal. Find the optimal one and print its modularity score.

>*Hint: Either just try a bunch of different label combinations or visualize the networks with node colors that correspond to community, so you can see what is optimal.*

#### Finding the best communities

OK, so we are now able to evaluate the quality of a partition. But how do we find the best partition? Modularity gives us a way to measure *how good* our partition is, but it does't tell of how to find the best one. For that we need some sort of algorithm. The *Louvain method* is that algorithm.

It works in the following steps:
1. Set every node to be its own community (initiate `c = [0, 1, 2, 3, 4, 5]`).
2. Compute the modularity.
3. Now pick a random node.
    1. For every neighbor it has, try giving it the neighbor's label, and compute the change in modularity.
    2. If any of those relabelings led to an increase in modularity, choose the relabeling with the greatest increase.
4. Repeat 2-3 until modularity ceases to increase for any relabelings.

> **Ex. 5.1.BONUS**: Implement the Louvain method, and show that it gives the labeling for `A`.

#### Partnership communities in the Marvel universe

Now let's work with some real data. Whip out the network you created last week, we will be using that again.

>**Ex. 5.1.4**: Remove all nodes that have zero local clustering coefficient. Print the number of nodes and edges in this new graph. I get 100 nodes and 140 edges.

> **Ex. 5.1.5**: Find the communities in this networks. Visualize it, coloring the nodes according to their label.

>*Hint: If you didn't implement the Louvain method in the bonus exercise above, you will need a package which can do it for you. Go ahead and install `python-louvain` by running `conda install -c auto python-louvain` in a terminal. After installation, import it with `import community`.*

### Part 2: Contagion

If you have a network where links represent interactions that can happen between actors, then you can use that as a model universe for simulating the spreading of a disease, a rumor, or some piece of information. There's a whole subfield of Network Science called Contagion Theory which deals with this, but the fundamental idea is actually quite simple:

> Nodes that are connected can pass disease/information with some probability and eventually things stabilize.

You can then make this more nuanced by *inventing* that nodes can only be infected if multiple neighbors are infected, that they can be cured and become resistent/susceptible with some probability, that they may die and exit the simulation creating a void in the network, that becoming infected makes them change their links, that godzilla walks through the door and eats everyone—the possibilities are endless and you decide which rules exist in the simulation universe.

> **Ex. 5.2.1**: The simplest spreading process is the *SI* model. Nodes in the network can either be healthy but **S**usceptible, or **I**nfected and non-susceptible. At each time-step, an *I*-node can infect an *S*-node with probability $p_I$. Below I have implemented such a simulation.
1. Comment the code abundantly, explaining that happing at each step.
2. Adapt the code so you can run the simulation multiple times and create a histogram of the fraction of the network infected, $F$, at the end of each simulation. Comment on this result. Could you have made a similar looking histogram in a different way?

In [418]:
import numpy as np

p_I = 0.1
n_iter = 200
G = yournetwork

I = set()
S = set(G.nodes())

patient0 = np.random.choice(list(S))

I.add(patient0)
S.remove(patient0)

for t in range(n_iter):
    for infected_node in list(I):
        neighbors = G.neighbors(infected_node)
        infected_neighbors = set([n for n in neighbors if np.random.random() < p_I])
        I |= infected_neighbors
        S -= infected_neighbors

> **Ex. 5.2.2**: Clusters are well known to mitigate the effects of contagion. This makes sense. If people only go in small circles, they can't really pass an infection to the global community. In the Marvel partnerhip network—as we have rendered it after removing non-clustered nodes—clusters are highly confined. Let's try to see how the infection fraction $F$ is changes when we break the clusters.
* Use the function `networkx.swap.double_edge_swap` to swap random edges in the network, driving it towards a more and more randomly connected (and less clustered) network. If `R` is the number of swaps made, then iterate over `for R in range(100)`, and measure the average $F$ (denoted $\left<F\right>$) of multiple SI simulations in each iteration.
* Make a scatter plot of $R$ vs. $\left<F\right>$.

>*Hint: Due to the way Python binds memory, inputting a copy of your network to the swapping function may save you some trouble.*

> **Ex. 5.2.3**: The *SI* process is admittedly very simple. The slightly more avanced *SIS* process is almost the same as the *SI*, only it allows nodes to become healthy and susceptible again. Where *SI* had only only one parameter—the infection probability, $p_I$—the *SIS* process requires another parameter, $p_S$, which is the probability that in a timestep a node becomes healthy and susceptible again. Swap enough edges in your network for the contagion to be able to reach every node.
1. Implement the *SIS* process. Make sure to comment abundantly, explaining your logic.
2. For $p_I=p_S=0.1$, make a time-series plot spanning over 200 timesteps which visualizes $F$ at each timestep.
3. When $p_S$ is close to 1, it means that nodes become healthy very easily. Intuitively this should mean that the $F$ curve goes to zero after only a few timesteps (contagion dies out). For $p_I=0.1$ find the smallest value of $p_S$ that still causes the $F$ to go to zero. Show the according timeseries plot.
4. Reversely, when $p_S$ is close to 0, the infected nodes are likely to stay infected and the whole network will become infected. For $p_I=0.1$ find approximately the smallest value of $p_S$ that is able to causes the $F$ curve to go to 1. Show the according timeseries plot.
5. Explain these results in your own words. Everyday language that a lay person can understand is much appreciated here.